# Step0: Install environment

In [3]:
import vapoursynth as vs
import havsfunc as haf
import mvsfunc as mvf
from vapoursynth import core
import awsmfunc as awsf

# Step1: Select BDMV Largest M2TS and its MLPS

In [ ]:
x265_filename=r"C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME.mkv"

# Step2: BDMV to MKV

# Step3: Transcode

In [1]:
import os, pyperclip, re, subprocess
from pymediainfo import MediaInfo
from datetime import datetime
from utils import *
crf=18.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"
remotecwd="~/compressor"
remotehost="amd"
PROCUCT_DEST_DIR=r"C:\Users\heber\Music"
MOVIE=r"C:\Users\heber\Music\Get.Out.2017.UHD.Blu-Ray.2160p.HEVC.REMUX.DTS-louco@PTH.mkv"

## get remux capture and crop black borders

In [6]:
get_captures("REMUX", MOVIE)

Capture REMUX


## H264: Test params

In [12]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

args=x264_stmt.replace(" --","\n --").split("\n")[1:]
args=sorted(args)
# print("\n".join(["x264 参数："]+args))

remote_x264stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat|{x264_stmt} 2>&1 | tee x264.log"{newline}'

print("==========\n远程执行编码 x264:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=YES{newline}'
cmd+=remote_x264stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
# cmd+=f'scp {remotehost}:~/264.mkv D:\\BT\\samples\\264.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264:
set MOVIE=C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
set TEST=YES
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat|x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log"
scp amd:~/264.mkv C:\Users\heber\Documents\Compressor\264.mkv
pause



## H264&H265

In [2]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"


remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x264+x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x264.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x264.log & '
cmd+=f'ssh {remotehost} "cat x265.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x265.log{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264+x265:
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log) >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 --qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-op

# H265 4K offline

In [3]:
x265_stmt=f'x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --psy-rdoq 1 --psy-rd 2  --bframes 11 \
--ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 \
--me 3 --subme 6 --merange 48 --analyze-src-pics \
--max-merge 5 --early-skip --b-intra \
--no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 \
--no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 \
--rdoq-level 2 --nr-inter 20 --nr-intra 10 \
--qg-size 32 \
--rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast \
--rdpenalty 0 --no-splitrd-skip \
--deblock -3:-3 --nr-inter 20 --nr-intra 10 \
--no-sao --no-sao-non-deblock --selective-sao 0 \
--no-opt-qp-pps --no-opt-ref-list-length-pps --repeat-headers \
--no-temporal-layers --no-amp --pools + \
--vbv-bufsize 160000 --vbv-maxrate 150000 --lookahead-slices 5 --input-depth 10 \
--hdr10 --hdr10-opt --colormatrix bt2020nc --colorprim bt2020 --transfer smpte2084 \
--master-display \'G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(10000000,0)\' --output 265.mkv -'
# --max-cll 922,239 
'''
BT.2020 	G(8500,39850)B(6550,2300)R(35400,14600)WP(15635,16450)L(40000000,50)
Display P3    G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(40000000,50)
'''

# with open("x265.sh","w") as f:
#     f.write(f'zstdcat ~/yrm/src|{x265_stmt} 2>&1 | tee x265.log')
# remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
# remote_convertcmd=f'scp {os.getcwd()}\\x265.sh {remotehost}:{remotecwd}'
    
remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x265.log|sed \'s/.*\\r//g\' " >{os. getcwd()}\\x265.log{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\Get.Out.2017.UHD.Blu-Ray.2160p.HEVC.REMUX.DTS-louco@PTH.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --psy-rdoq 1 --psy-rd 2  --bframes 11 --ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 --me 3 --subme 6 --merange 48 --analyze-src-pics --max-merge 5 --early-skip --b-intra --no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 --no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 --rdoq-level 2 --nr-inter 20 --nr-intra 10 --qg-size 32 --rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast --rdpenalty 0 --no-splitrd-skip --deblock -3:-3 --nr-inter 20 --nr-intra 10 --no-sao --no-sao-non-d

## H264 & H265 offline

In [5]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

remote_stmt=f'zstdcat ~/yrm/src| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'

with open("x26x.sh","w") as f:
    f.write(remote_stmt)
remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
remote_convertcmd=f'scp {os.getcwd()}\\x26x.sh {remotehost}:{remotecwd}'

print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_x265stmt+newline
cmd+=remote_convertcmd+newline
cmd+=f'pause{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/265.mkv {os. getcwd()}\\265.mkv{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T8|ssh ubuntu20local "cat>~/y4m/src"
scp C:\Users\heber\Documents\Compressor\x26x.sh ubuntu20local:~/compressor
pause
scp ubuntu20local:~/compressor/265.mkv C:\Users\heber\Documents\Compressor\265.mkv



# Step4: Remux MKV
## x264

In [17]:
# title="".join(re.findall(r'([^\\]*\d{4}p)', MOVIE))
# NEWMOVIE=f'{os.getcwd()}\\{title}.BluRay.x264.DTS-PTH.mkv'
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX.AVC", "x264")
x264_foldername=PROCUCT_DEST_DIR+"\\"+x264_filename.replace(".mkv",'')
NEWMOVIE=f'{x264_foldername}\\{x264_filename}'
print(x264_filename)
cmd=f'mkdir {x264_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\264.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\264.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv
即将执行: 
   [剩] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv
 + [视] C:\Users\heber\Documents\Compressor\264.mkv
-> [全] C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv
mkdir C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome\Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome.mkv" -D ( "C:\Users\heber\Music\Forrest.Gump.1994.Remastered.1080p.Blu-ray.REMUX.AVC.Atmos.TrueHD.7.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\264.mkv" )


In [18]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)
None

In [22]:
generate_videoinfo(f'{x264_foldername}\\{x264_filename}',"x264.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x264.Atmos.TrueHD.7.1-louco@PTHome
RELEASE DATE..............: 2021-01-12
RELEASE SiZE..............: 17.591 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 2 h 22 min
ViDEO CODEC...............: x264 High@L4.1 @ 13.35Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)


.x264.Info
x264 [info]: profile High, level 4.1, 4:2:0, 8-bit
x264 [info]: frame I:1182  Avg QP:17.00  size:270311
x264 [info]: frame P:49116 Avg QP:18.81  size:113585
x264 [info]: frame B:154121 Avg QP:20.83  size: 54055
x264 [info]: 

In [51]:
get_captures("x264", x264_filename)

Capture x264


## x265

In [14]:
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_foldername=PROCUCT_DEST_DIR+"\\"+x265_filename.replace(".mkv",'')
NEWMOVIE=f'{x265_foldername}\\{x265_filename}'
x265_filename=NEWMOVIE
cmd=f'mkdir {x265_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\265.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\265.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

即将执行: 
   [剩] C:\Users\heber\Music\菠萝快车.Pineapple.Express.2008.BluRay.2160p.x265.10bit.HDR.mUHD-FRDS\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.REMUX.TrueHD.7.1.mkv
 + [视] C:\Users\heber\Documents\Compressor\265.mkv
-> [全] C:\Users\heber\Music\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1.mkv
mkdir C:\Users\heber\Music\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1 ^& C:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "C:\Users\heber\Music\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1.mkv" -D ( "C:\Users\heber\Music\菠萝快车.Pineapple.Express.2008.BluRay.2160p.x265.10bit.HDR.mUHD-FRDS\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.REMUX.TrueHD.7.1.mkv" )    -A -S -T -M -B --no-chapters ( "C:\Users\heber\Documents\Compressor\265.mkv" )


In [11]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)

CompletedProcess(args='start cmd /c mkdir C:\\Users\\heber\\Music\\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1 ^& C:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "C:\\Users\\heber\\Music\\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1\\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1.mkv" -D ( "C:\\Users\\heber\\Music\\菠萝快车.Pineapple.Express.2008.BluRay.2160p.x265.10bit.HDR.mUHD-FRDS\\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.REMUX.TrueHD.7.1.mkv" )    -A -S -T -M -B --no-chapters ( "C:\\Users\\heber\\Documents\\Compressor\\265.mkv" ) ^& pause', returncode=0)

In [15]:
generate_videoinfo(x265_filename,"x265.log")
None

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1
RELEASE DATE..............: 2021-01-19
RELEASE SiZE..............: 10.264 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 1 h 52 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 8.55Mb/s
RESOLUTiON................: 3840x1328
DiSPLAY ASPECT RATiO......: 2.892
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4482.23Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : English(en)
SUBTiTLES.................: S_TEXT/ASS : Chinese(简体中文)


.x265.Info
[/quote]


In [13]:
print(x265_filename)
get_captures("x265", x265_filename)

Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1.mkv
Capture x265


In [34]:
get_mediainfo(x265_filename)
None

Copy Mediainfo


# Step5 发布

In [3]:
def get_bbcode(file, src="",logname=""):
    print("Title:", file.replace("."," "))
#     由ZERO13@PTHome字幕组完成校对、调整。
    picbed='''
[url=https://sm.ms/image/OgQbM7si1YImXye][img]https://i.loli.net/2021/01/19/OgQbM7si1YImXye.jpg[/img][/url]
[url=https://sm.ms/image/ErSyogQaxX5sUfq][img]https://i.loli.net/2021/01/19/ErSyogQaxX5sUfq.jpg[/img][/url]
[url=https://sm.ms/image/aN9gFDyKH7OGIMo][img]https://i.loli.net/2021/01/19/aN9gFDyKH7OGIMo.jpg[/img][/url]
[url=https://sm.ms/image/5MPY7ox6wXE4bdV][img]https://i.loli.net/2021/01/19/5MPY7ox6wXE4bdV.jpg[/img][/url]
[url=https://sm.ms/image/yJ7nuXhsBe9NCgf][img]https://i.loli.net/2021/01/19/yJ7nuXhsBe9NCgf.jpg[/img][/url]
[url=https://sm.ms/image/OTfmvWFHbYtQBha][img]https://i.loli.net/2021/01/19/OTfmvWFHbYtQBha.jpg[/img][/url]
    '''
    if "REMUX" in file:
        picbed='''
[url=https://sm.ms/image/FjRpzKqL2rkdGCA][img]https://i.loli.net/2021/01/06/FjRpzKqL2rkdGCA.jpg[/img][/url]
[url=https://sm.ms/image/DR79FMJIpYS1O42][img]https://i.loli.net/2021/01/06/DR79FMJIpYS1O42.jpg[/img][/url]
[url=https://sm.ms/image/nHLCScypZ4wVP7N][img]https://i.loli.net/2021/01/06/nHLCScypZ4wVP7N.jpg[/img][/url]
[url=https://sm.ms/image/LilaOG3cDn7qzQg][img]https://i.loli.net/2021/01/06/LilaOG3cDn7qzQg.jpg[/img][/url]
[url=https://sm.ms/image/ZYVtgW8Eo45McDG][img]https://i.loli.net/2021/01/06/ZYVtgW8Eo45McDG.jpg[/img][/url]
[url=https://sm.ms/image/sHNcvD1mP8bCkoB][img]https://i.loli.net/2021/01/06/sHNcvD1mP8bCkoB.jpg[/img][/url]
    '''
    bbcode='''
[quote][size=5][color=RoyalBlue]原盘来自TTG，字幕素材来TTG-R3
字幕转载务必注意礼节，请保留原作者信息，谢绝二次提取修改！
在此感谢各位原作者及分享者！如有侵权请联系删除！[/color][/size][/quote]
[img]https://pic.imgdb.cn/item/5ff590053ffa7d37b3bd4495.jpg[/img]

◎译　　名　菠萝快车 / 菠萝快递 / 菠萝特快
◎片　　名　Pineapple Express
◎年　　代　2008
◎产　　地　美国
◎类　　别　喜剧 / 动作 / 惊悚 / 犯罪 / 冒险
◎语　　言　英语 / 粤语 / 韩语
◎上映日期　2008-08-06(美国)
◎IMDb评分  7.0/10 from 298101 users
◎IMDb链接  http://www.imdb.com/title/tt0910936
◎豆瓣评分　7.0/10 from 14777 users
◎豆瓣链接　https://movie.douban.com/subject/2074035/
◎片　　长　111 分钟
◎导　　演　David Gordon Green
◎编　　剧　Seth Rogen / 埃文·戈德堡 Evan Goldberg / 贾德·阿帕图 Judd Apatow
◎主　　演　塞斯·罗根 Seth Rogen
　　　　　  詹姆斯·弗兰科 James Franco
　　　　　  丹尼·麦克布耐德 Danny McBride
　　　　　  凯文·考利甘 Kevin Corrigan
　　　　　  克雷格·罗宾森 Craig Robinson
　　　　　  盖瑞·科尔 Gary Cole
　　　　　  罗西·培瑞兹 Rosie Perez
　　　　　  小艾德·博格里 Ed Begley Jr.
　　　　　  诺拉·邓恩 Nora Dunn
　　　　　  安布尔·赫德 Amber Heard
　　　　　  乔·洛·特鲁格里奥 Joe Lo Truglio
　　　　　  亚瑟·纳平泰克 Arthur Napiontek
　　　　　  克利奥·金 Cleo King
　　　　　  比尔·哈德尔 Bill Hader
　　　　　  詹姆斯·瑞马尔 James Remar


◎标　　签　喜剧 | 美国 | JamesFranco | 黑色幽默 | 犯罪 | 美国电影 | 2008 | SethRogen

◎简　　介  

　　戴尔（塞斯•罗根 Seth Rogen 饰）有一份令自己和别人都生厌的工作——传票递送员，高中生女友安吉的帅气男同学们又时时令戴尔如坐针毡，于是抽大麻成了他最好的寄托甚至是信仰。戴尔定期在小混混索尔（詹姆斯•弗兰科 James Franco 饰）的家中购买大麻，索尔视其为朋友，推销给他最好的大麻“菠萝快车”，可戴尔只拿对方当做比自己更失败的落寞者。当晚戴尔偶然撞见当地大毒贩瑞德和一名女警联手杀人，慌不择路逃跑的戴尔将菠萝快车这种罕见大麻遗落在现场，担心被瑞德杀人灭口的戴尔和索尔只得选择逃亡，但他们谨慎又笨拙的隐匿方式只是将更多人卷入这场风波，而逃避性格的两人，在这次事件中渐渐学会正视自己的弱点……

◎获奖情况  

　　第66届金球奖(2009)
　　电影类 音乐/喜剧片最佳男主角(提名) 詹姆斯·弗兰科

　　第18届MTV电影奖(2009)
　　MTV电影奖 最佳喜剧表演(提名) 詹姆斯·弗兰科
　　MTV电影奖 最佳打斗(提名) 詹姆斯·弗兰科 / 塞斯·罗根 / 丹尼·麦克布莱德

　　第11届美国青少年选择奖(2009)
　　最佳喜剧片男演员(提名) 塞斯·罗根
　　Choice Movie: Bromantic Comedy(提名)
[img]https://images2.imgbox.com/04/6b/Ggp5ReQb_o.png[/img]

'''+f'{generate_videoinfo(file,logname,src=src)}\n{picbed}\n{get_mediainfo(file)}'
    pyperclip.copy(bbcode)
SRC=r"Pineapple Express 2008 2160p UHD Blu-ray HEVC Atmos TrueHD 7.1-DIY@TTG".replace(" ",".")

In [19]:
from utils import *
REMUX=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH.mkv"
get_bbcode(REMUX,src=SRC)

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p REMUX AVC DTS-HD 5 1-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p REMUX AVC DTS-HD 5 1-louco@PTH mkv


FileNotFoundError: C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.REMUX.AVC.DTS-HD 5.1-louco@PTH.mkv

In [47]:
X264FINAL=r"C:\Users\heber\Music\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH\A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH.mkv"
get_bbcode(X264FINAL,src=SRC,logname="x264.log")

Title: C:\Users\heber\Music\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH\A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p x264 DTS-louco@PTH mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: A.Good.Old.Fashioned.Orgy.2011.Blu-ray.RUS.1080p.x264.DTS-louco@PTH
RELEASE DATE..............: 2021-01-06
RELEASE SiZE..............: 11.457 GB
SOURCE....................: A Good Old Fashioned Orgy 2011 Blu-ray RUS 1080p AVC DTS-HD 5.1-hdsng@HDSky

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x264 High@L4.1 @ 16.47Mb/s
RESOLUTiON................: 1920x1036
DiSPLAY ASPECT RATiO......: 1.853
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Russian DTS 5.1 @ 768.0Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(zhs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(zht)
SUBTiTLES.................: S_HDMV/PGS : Russian(ru)


.x264.Inf

In [4]:
X265FINAL=r"C:\Users\heber\Music\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1-louco@PTH\Pineapple.Express.2008.BluRay.2160p.UHD.HEVC.HDR.x265.10bit.TrueHD.7.1-louco@PTH.mkv"
# get_captures("x265",X265FINAL)
get_bbcode(X265FINAL,src=SRC,logname="x265.log")

Title: C:\Users\heber\Music\Pineapple Express 2008 BluRay 2160p UHD HEVC HDR x265 10bit TrueHD 7 1-louco@PTH\Pineapple Express 2008 BluRay 2160p UHD HEVC HDR x265 10bit TrueHD 7 1-louco@PTH mkv


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

# 备忘录